# AI2S Hackaton
## Team Assacro

**Optimization Problem(s)**

# Import data and libraries

In [2]:
import numpy as np 
import pandas as pd

In [3]:

df_target = pd.read_csv("../data/02_input_target.csv")


df_capacity = pd.read_csv("../data/02_input_capacity.csv")

df_capacity['Monthly Capacity'] *= 5

N_COUNTRIES = df_target['Country'].unique().shape[0]
N_PRODUCTS = df_target['Product'].unique().shape[0]
N_MONTHS = 12

In [4]:
import gurobipy as gp
from gurobipy import GRB

In [5]:
m = gp.Model("mip1")

Set parameter Username
Set parameter LicenseID to value 2640110
Academic license - for non-commercial use only - expires 2026-03-20


# Define Variables

$$
x_{ijk}: \text{ Amount of products } j \text{ allocated in city }i \text{ during month }k
$$

In [6]:
# define first batch of variables
X = {}
for i in range(N_COUNTRIES):
    for j in range(N_PRODUCTS):
        for k in range(N_MONTHS):
            X[i,j,k] = m.addVar(lb = 0,vtype=GRB.INTEGER, name=f"x{i}{j}{k}")

m.update()

In [7]:
# Generate hashmap to convert country -> i, product -> j, month -> k

In [8]:
hashmap_months = {
    0: "Jan",
    1: "Feb",
    2: "Mar",
    3: "Apr",
    4: "May",
    5: "Jun",
    6: "Jul",
    7: "Aug",
    8: "Sep",
    9: "Oct",
    10: "Nov",
    11: "Dec"
}


In [9]:
hashmap_products = dict(enumerate(df_target['Product'].unique()))

In [10]:
hashmap_countries = dict(enumerate(df_target['Country'].unique()))

In [11]:
def get_names(i,j,k):
    return (hashmap_countries[i], hashmap_products[j], hashmap_months[k])

In [12]:
hashmap_months_inv = {v: k for k, v in hashmap_months.items()}
hashmap_products_inv = {v: k for k, v in hashmap_products.items()}
hashmap_countries_inv = {v: k for k, v in hashmap_countries.items()}

In [13]:
df_capacity['country_code'] = df_capacity['Country'].map(hashmap_countries_inv)

# Define Costraints
$$
\forall i, k; \sum_{j} x_{ijk} \leq c_{i}
$$

In [14]:
# constraint 1: capacity costraint

for i in range(N_COUNTRIES):
    for k in range(N_MONTHS):
        S = 0
        for j in range(0, N_PRODUCTS):
            S += X[i,j,k]

        sub = df_capacity[['Monthly Capacity', 'country_code']]
        c_ijk = sub[sub['country_code'] == i].loc[:, 'Monthly Capacity'].values[0]

        m.addConstr(S <= c_ijk)

In [15]:
m.update()

In [16]:
df_target['Month_'] = df_target['Month'].str[:3] # 2004 is redundant

In [17]:
df_target

,Country,Product,Month,Quantity,Month_
0,Japan,MorningMint,Jan2004,0,Jan
1,Japan,MorningMint,Feb2004,0,Feb
2,Japan,MorningMint,Mar2004,0,Mar
3,Japan,MorningMint,Apr2004,0,Apr
4,Japan,MorningMint,May2004,0,May
...,...,...,...,...,...
11995,Russia,HydratingHoney Pet Wipes,Aug2004,1129,Aug
11996,Russia,HydratingHoney Pet Wipes,Sep2004,1143,Sep
11997,Russia,HydratingHoney Pet Wipes,Oct2004,1249,Oct
11998,Russia,HydratingHoney Pet Wipes,Nov2004,1058,Nov


In [18]:
df_target['country_code'] = df_target['Country'].map(hashmap_countries_inv)
df_target['month_code'] = df_target['Month_'].map(hashmap_months_inv)
df_target['product_code'] = df_target['Product'].map(hashmap_products_inv)

# create mappings

In [19]:
df_target

,Country,Product,Month,Quantity,Month_,country_code,month_code,product_code
0,Japan,MorningMint,Jan2004,0,Jan,0,0,0
1,Japan,MorningMint,Feb2004,0,Feb,0,1,0
2,Japan,MorningMint,Mar2004,0,Mar,0,2,0
3,Japan,MorningMint,Apr2004,0,Apr,0,3,0
4,Japan,MorningMint,May2004,0,May,0,4,0
...,...,...,...,...,...,...,...,...
11995,Russia,HydratingHoney Pet Wipes,Aug2004,1129,Aug,9,7,99
11996,Russia,HydratingHoney Pet Wipes,Sep2004,1143,Sep,9,8,99
11997,Russia,HydratingHoney Pet Wipes,Oct2004,1249,Oct,9,9,99
11998,Russia,HydratingHoney Pet Wipes,Nov2004,1058,Nov,9,10,99


In [20]:
df_target_sum = df_target[['month_code', 'country_code', 'Quantity']].groupby(['country_code', 'month_code']).sum() 
# Define sum of country (and month) on each product produced

In [21]:
my_df_target = df_target[['country_code', 'product_code', 'month_code', 'Quantity']]


$$
x_{i,j,k} \geq d_{i,j,k}
$$

In [22]:
# Costraints 2: satisfy market demands
for i in range(N_COUNTRIES):
    for k in range(N_MONTHS):
        for j in range(0, N_PRODUCTS):
            x_ijk = my_df_target[(my_df_target['country_code'] == i) & (my_df_target['product_code'] == j) & (my_df_target['month_code'] == k)]['Quantity'].values[0] 
            m.addConstr(X[i,j,k] >= x_ijk)


In [23]:
m.update()

# Define Objective Fun

$$ \min z = \sum_{i,j,k} x_{i,j,k} $$

In [24]:
S_obj = 0
for i in range(N_COUNTRIES):
    for j in range(N_PRODUCTS):
        for k in range(N_MONTHS):
            S_obj += X[i,j,k] 

In [25]:
S_obj

<gurobi.LinExpr: x000 + x001 + x002 + x003 + x004 + x005 + x006 + x007 + x008 + x009 + x0010 + x0011 + x010 + x011 + x012 + x013 + x014 + x015 + x016 + x017 + x018 + x019 + x0110 + x0111 + x020 + x021 + x022 + x023 + x024 + x025 + x026 + x027 + x028 + x029 + x0210 + x0211 + x030 + x031 + x032 + x033 + x034 + x035 + x036 + x037 + x038 + x039 + x0310 + x0311 + x040 + x041 + x042 + x043 + x044 + x045 + x046 + x047 + x048 + x049 + x0410 + x0411 + x050 + x051 + x052 + x053 + x054 + x055 + x056 + x057 + x058 + x059 + x0510 + x0511 + x060 + x061 + x062 + x063 + x064 + x065 + x066 + x067 + x068 + x069 + x0610 + x0611 + x070 + x071 + x072 + x073 + x074 + x075 + x076 + x077 + x078 + x079 + x0710 + x0711 + x080 + x081 + x082 + x083 + x084 + x085 + x086 + x087 + x088 + x089 + x0810 + x0811 + x090 + x091 + x092 + x093 + x094 + x095 + x096 + x097 + x098 + x099 + x0910 + x0911 + x0100 + x0101 + x0102 + x0103 + x0104 + x0105 + x0106 + x0107 + x0108 + x0109 + x01010 + x01011 + x0110 + x0111 + x0112 + x

In [26]:
m.setObjective(S_obj, GRB.MINIMIZE)

In [27]:
m.update()

In [28]:
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1240P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 12120 rows, 12000 columns and 24000 nonzeros
Model fingerprint: 0x2efedf76
Variable types: 0 continuous, 12000 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+05]
Presolve removed 12000 rows and 12000 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [29]:
m.remove(m.getConstrs())
m.update()

# Define Another Batch of Variables

$$
T_{i,i', j, k}: \text{ Amount of product }j\text{ transferred from country }i \text{ to country } i' \text{ in month }k
$$

In [30]:
T = {}
for i in range(N_COUNTRIES):
    for i_ in range(N_COUNTRIES):
        for j in range(N_PRODUCTS):
            for k in range(N_MONTHS):
                T[i, i_, j, k] = m.addVar(lb=0, vtype=GRB.INTEGER, name=f'T{i}{i_}{j}{k}')

m.update()

In [31]:
def get_val(i,j,k):
    return my_df_target[(my_df_target['country_code'] == i) & (my_df_target['product_code'] == j) & (my_df_target['month_code'] == k)]['Quantity'].values[0] 

# Redefine and Add Costraints

$$
\sum_{j} x_{ijk} \leq c_i
$$

In [32]:
# capacity costraints
for i in range(N_COUNTRIES):
    for k in range(N_MONTHS):
        S = 0
        for j in range(0, N_PRODUCTS):
            S += X[i,j,k]
            
        sub = df_capacity[['Monthly Capacity', 'country_code']]
        c_ijk = sub[sub['country_code'] == i].loc[:, 'Monthly Capacity'].values[0]

        m.addConstr(S <= c_ijk , name=f"{hashmap_countries[i]} capacity")

In [33]:
m.update()

$$
x_{ijk} + \sum_{o \neq i} T_{oijk} \geq d_{ijk}
$$

In [34]:
# demand costraints
for i in range(N_COUNTRIES):
    for j in range(N_PRODUCTS):
        for k in range(N_MONTHS):
            S_t = 0
            for o in range(N_COUNTRIES):
                if o == i:
                    continue
                S_t += (T[o, i, j, k])

            S_t2 = 0
            for d in range(N_COUNTRIES):
                if d == i:
                    continue
                S_t2 += T[i,d,j,k]

            S_t += X[i,j,k]
            S_t - S_t2

            x_ijk = get_val(i,j,k)

            m.addConstr(S_t == x_ijk, name=f"{hashmap_countries[i]} demand of {hashmap_products[j]}")

m.update()

$$
x_{ijk} - \sum_{d \neq i} T_{idjk} \geq 0
$$

In [35]:
# transfer costraints
for i in range(N_COUNTRIES):
    for k in range(N_MONTHS):
        for j in range(N_PRODUCTS):
            S_2 = 0

            S_1 = X[i,j,k]
            for d in range(N_COUNTRIES):
                if d == i:
                    continue 
                S_2 += T[i, d,j, k]

            m.addConstr(S_1 - S_2  >= 0, name=f"{hashmap_countries} country can send")

m.update()

# Redefine Target Fun

$$
\min z = 0.01(\sum_{ijk} x_{ijk}) + \sum_{odjk} T_{odjk}
$$

In [36]:
S_X = 0
for i in range(N_COUNTRIES):
    for j in range(N_PRODUCTS):
        for k in range(N_MONTHS):
            S_X += X[i,j,k]

S_S = 0
for o in range(N_COUNTRIES):
    for d in range(N_COUNTRIES):
        if o != d:
            for j in range(N_PRODUCTS):
                for k in range(N_MONTHS):
                    S_S += T[o,d,j,k]

S_tot1 = S_X*0.01 + S_S 

m.setObjective(S_tot1, GRB.MINIMIZE)

In [37]:
m.update()

m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1240P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 24120 rows, 132000 columns and 252000 nonzeros
Model fingerprint: 0xad6efd98
Variable types: 0 continuous, 132000 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+05]
Found heuristic solution: objective 3364326.1700
Presolve removed 23246 rows and 130037 columns
Presolve time: 0.27s
Presolved: 874 rows, 1963 columns, 4274 nonzeros
Found heuristic solution: objective 1856807.6300
Variable types: 0 continuous, 1963 integer (0 binary)
Found heuristic solution: objective 1855233.5300

Root relaxation: objective 1.829504e+06, 630 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    

# Export Second Submission

In [38]:
T_vals = {}

In [39]:
hashmap_countries

{0: 'Japan',
 1: 'Australia',
 2: 'Brazil',
 3: 'Egypt',
 4: 'Italy',
 5: 'South Africa',
 6: 'France',
 7: 'India',
 8: 'United Kingdom',
 9: 'Russia'}

In [40]:
for o in range(N_COUNTRIES):
    for d in range(N_COUNTRIES):
        for j in range(N_PRODUCTS):
            for k in range(N_MONTHS):
                T_vals[
                    hashmap_countries[o],
                    hashmap_countries[d],
                    hashmap_products[j],
                    hashmap_months[k]
                ] = T[o,d,j,k].X

In [41]:
X_vals = {}
for i in range(N_COUNTRIES):
    for j in range(N_PRODUCTS):
        for k in range(N_MONTHS):
            X_vals[
                hashmap_countries[i],
                hashmap_products[j],
                hashmap_months[k]
            ] = X[i,j,k].X

In [42]:
df_shipments = pd.DataFrame(
    [(k[0], k[1], k[2], k[3], v) for k, v in T_vals.items()],
    columns=['Origin', 'Destination', 'Product', 'Month', 'Quantity']
)

In [43]:
df_proposal =pd.DataFrame(
    [(k[0], k[1], k[2], v) for k, v in X_vals.items()],
    columns=['Country', 'Product', 'Month', 'Quantity']
)

In [44]:
df_shipments['Month'] = df_shipments['Month'] + "2004"
df_proposal['Month'] = df_proposal['Month'] + "2004"


In [45]:
df_proposal[df_proposal['Quantity'] != 0]

,Country,Product,Month,Quantity
12,Japan,FreshStart Toothpaste,Jan2004,1471.0
13,Japan,FreshStart Toothpaste,Feb2004,1135.0
14,Japan,FreshStart Toothpaste,Mar2004,1527.0
15,Japan,FreshStart Toothpaste,Apr2004,593.0
16,Japan,FreshStart Toothpaste,May2004,1664.0
...,...,...,...,...
11995,Russia,HydratingHoney Pet Wipes,Aug2004,1129.0
11996,Russia,HydratingHoney Pet Wipes,Sep2004,1143.0
11997,Russia,HydratingHoney Pet Wipes,Oct2004,1249.0
11998,Russia,HydratingHoney Pet Wipes,Nov2004,1058.0


In [46]:
df_shipments[df_shipments['Quantity']>0]

,Origin,Destination,Product,Month,Quantity
7212,Japan,France,FreshStart Toothpaste,Jan2004,879.0
7213,Japan,France,FreshStart Toothpaste,Feb2004,872.0
7214,Japan,France,FreshStart Toothpaste,Mar2004,359.0
7215,Japan,France,FreshStart Toothpaste,Apr2004,592.0
7216,Japan,France,FreshStart Toothpaste,May2004,924.0
...,...,...,...,...,...
118795,Russia,United Kingdom,HydratingHoney Pet Wipes,Aug2004,1129.0
118796,Russia,United Kingdom,HydratingHoney Pet Wipes,Sep2004,1143.0
118797,Russia,United Kingdom,HydratingHoney Pet Wipes,Oct2004,1128.0
118798,Russia,United Kingdom,HydratingHoney Pet Wipes,Nov2004,317.0


In [47]:
df_shipments[df_shipments['Origin'] == df_shipments['Destination']]['Quantity'].describe()

count    12000.0
mean         0.0
std          0.0
min         -0.0
25%         -0.0
50%         -0.0
75%          0.0
max         -0.0
Name: Quantity, dtype: float64

In [48]:
df_shipments.to_csv("my_shipments.csv", index=False)

In [49]:
df_proposal.to_csv("my_proposals.csv", index=False)

# Tweak Problem for taking weights (costs) into account

In [50]:
# Step 3: Tweak objective function to take production costs and weights into account

In [51]:
df_shipments_weights = pd.read_csv('../data/02_03_input_shipmentsCost_example.csv')
df_production_costs = pd.read_csv('../data/03_input_productionCost.csv')

In [52]:
# Define dictionary of shipment weights
shipment_weights = {}

for o in range(N_COUNTRIES):
    for d in range(N_COUNTRIES):
        origin = hashmap_countries[o]
        destination = hashmap_countries[d]

        w = df_shipments_weights[(df_shipments_weights['Origin'] == origin) & (df_shipments_weights['Destination'] == destination)]['Unit Cost'].values
        if w.shape[0] != 1:
            shipment_weights[o, d] = -1
        else:
            shipment_weights[o,d] = w[0]
        pass

$$
(\sum_{j,k} T_{o,d,j,k})\text{ct}_{o,d} \geq 0
$$

In [53]:
# Add new constraint: shipments must be possible (more than 0)
for o in range(N_COUNTRIES):
    for d in range(N_COUNTRIES):
        if o == d:
            continue
        else:
            S = 0
            for j in range(N_PRODUCTS):
                for k in range(N_MONTHS):
                    S += T[o, d, j, k]

            S *= shipment_weights[o, d]

            m.addConstr(S >= 0, name=f"{hashmap_countries[o]}->{hashmap_countries[d]} | {j}{k}")

m.update()

In [54]:
def get_production_cost(i,j): # aux. function
    country = hashmap_countries[i]
    product = hashmap_products[j]

    return df_production_costs[(df_production_costs['Country'] == country) & (df_production_costs['Product'] == product)]['Unit Cost'].values[0]

$$
\min z = \sum_{o \neq d, j, k} (T_{o,d,j,k} \text{ct}_{o,d}) + \sum_{i,j,k} x_{i,j,k} \text{cp}_j
$$

In [55]:
# define obj. function
S_shipment = 0
S_prod = 0

for o in range(N_COUNTRIES):
    for d in range(N_COUNTRIES):
        if o != d:
            for j in range(N_PRODUCTS):
                for k in range(N_MONTHS):
                    S_shipment += T[o,d,j,k]*shipment_weights[o,d]

for i in range(N_COUNTRIES):
    for j in range(N_PRODUCTS):
        for k in range(N_MONTHS):
            S_prod += X[i,j,k]*get_production_cost(i,j)

S_tot = S_shipment + S_prod

m.setObjective(S_tot, GRB.MINIMIZE)

In [56]:
m.update()


In [57]:
m.reset()

Discarded solution information


In [58]:
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1240P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 24210 rows, 132000 columns and 360000 nonzeros
Model fingerprint: 0xc36c1671
Variable types: 0 continuous, 132000 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [3e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+05]
Found heuristic solution: objective 1.121716e+08
Presolve removed 23325 rows and 129466 columns
Presolve time: 0.88s
Presolved: 885 rows, 2534 columns, 5449 nonzeros
Found heuristic solution: objective 8.577974e+07
Variable types: 0 continuous, 2534 integer (0 binary)
Found heuristic solution: objective 8.563163e+07

Root relaxation: objective 8.403150e+07, 779 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    

# Export Last Submission

In [59]:
T_vals = {}
X_vals = {}

for o in range(N_COUNTRIES):
    for d in range(N_COUNTRIES):
        for j in range(N_PRODUCTS):
            for k in range(N_MONTHS):
                T_vals[
                    hashmap_countries[o],
                    hashmap_countries[d],
                    hashmap_products[j],
                    hashmap_months[k]
                ] = T[o,d,j,k].X

for i in range(N_COUNTRIES):
    for j in range(N_PRODUCTS):
        for k in range(N_MONTHS):
            X_vals[
                hashmap_countries[i],
                hashmap_products[j],
                hashmap_months[k]
            ] = X[i,j,k].X

In [60]:
df_shipments = pd.DataFrame(
    [(k[0], k[1], k[2], k[3], v) for k, v in T_vals.items()],
    columns=['Origin', 'Destination', 'Product', 'Month', 'Quantity']
)
df_proposal =pd.DataFrame(
    [(k[0], k[1], k[2], v) for k, v in X_vals.items()],
    columns=['Country', 'Product', 'Month', 'Quantity']
)
df_shipments['Month'] = df_shipments['Month'] + "2004"
df_proposal['Month'] = df_proposal['Month'] + "2004"
df_shipments.to_csv("my_shipments_3.csv", index=False)
df_proposal.to_csv("my_proposals_3.csv", index=False)